In [2]:
import mne
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np

In [5]:
file_path = r'C:\Users\omarh\Documents\GaTech\Project-InnerSpeech\Data\ds004196\sub-01\ses-EEG\eeg\sub-01_ses-EEG_task-inner_eeg.bdf'
eeg_data = mne.io.read_raw_bdf(file_path, preload=True)

channels_tsv_path = r'C:\Users\omarh\Documents\GaTech\Project-InnerSpeech\Data\ds004196\sub-01\ses-EEG\eeg\sub-01_ses-EEG_task-inner_channels.tsv'
channels_info = pd.read_csv(channels_tsv_path, sep='\t')

events_tsv_path = r'C:\Users\omarh\Documents\GaTech\Project-InnerSpeech\Data\ds004196\sub-01\ses-EEG\eeg\sub-01_ses-EEG_task-inner_events.tsv'
events_info = pd.read_csv(events_tsv_path, sep='\t')

print(events_info[:5])

Extracting EDF parameters from C:\Users\omarh\Documents\GaTech\Project-InnerSpeech\Data\ds004196\sub-01\ses-EEG\eeg\sub-01_ses-EEG_task-inner_eeg.bdf...
BDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 696319  =      0.000 ...  1359.998 secs...
   onset  duration trial_type
0      0    3999.0        six
1   3999    4000.0        six
2   7999    4000.0       four
3  11999    4000.0        ten
4  15999    4000.0       four


In [4]:
# Sort data into their channels
available_channels = set(eeg_data.ch_names).intersection(set(channels_info['name']))
eeg_data.pick_channels(list(available_channels))
montage = mne.channels.make_standard_montage('biosemi64')
eeg_data.set_montage(montage, on_missing='ignore')

NOTE: pick_channels() is a legacy function. New code should use inst.pick(...).


<RawEDF | sub-01_ses-EEG_task-inner_eeg.bdf, 60 x 696320 (1360.0 s), ~318.8 MB, data loaded>

In [19]:
# Function used to bin data
events_info['onset'] = events_info['onset'] / 1000
events_info['duration'] = events_info['duration'] / 1000

def bin_eeg_data(eeg_data, events_info):
    binned_data = []

    for _, event in events_info.iterrows():
        onset = event['onset']
        duration = event['duration']
        label = event['trial_type']
        
        start_sample = int(onset * eeg_data.info['sfreq'])
        stop_sample = int((onset + duration) * eeg_data.info['sfreq'])

        event_data, _ = eeg_data.get_data(start=start_sample, stop=stop_sample)

        binned_data.append({
            'label': label,
            'data': event_data
        })

    return binned_data

In [21]:
print(type(eeg_data))

<class 'mne.io.edf.edf.RawEDF'>


In [20]:
binned_eeg_data = bin_eeg_data(eeg_data, events_info)

for i, binned in enumerate(binned_eeg_data):
    print(f"Trial {i + 1}: Type = {binned['label']}, Data Shape = {binned['data'].shape}")

ValueError: too many values to unpack (expected 2)